In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import datetime
import requests
import os
import zipfile


### Wrangle Data

#### Gather, Assess, then Clean Data before moving on to exploratory analysis

#### Gathering

In [2]:
df = pd.read_csv('201810-fordgobike-tripdata.csv')

#### Assessing

In [3]:
df.shape

(201458, 14)

In [4]:
df.sample(5)

,duration_sec,start_time,end_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,bike_share_for_all_trip
198392,321,2018-10-01 11:56:17.3490,2018-10-01 12:01:39.1120,22.0,Howard St at Beale St,37.789756,-122.394643,21.0,Montgomery St BART Station (Market St at 2nd St),37.789625,-122.400811,793,Subscriber,No
164067,712,2018-10-06 00:05:58.2290,2018-10-06 00:17:51.1350,323.0,Broadway at Kearny,37.798014,-122.405950,3.0,Powell St BART Station (Market St at 4th St),37.786375,-122.404904,1837,Customer,No
182132,1360,2018-10-03 16:30:20.1340,2018-10-03 16:53:00.6870,24.0,Spear St at Folsom St,37.789677,-122.390428,107.0,17th St at Dolores St,37.763015,-122.426497,527,Subscriber,No
38108,432,2018-10-25 17:57:04.4800,2018-10-25 18:04:16.8900,58.0,Market St at 10th St,37.776619,-122.417385,98.0,Valencia St at 16th St,37.765052,-122.421866,3224,Subscriber,No
119898,338,2018-10-12 18:50:08.9730,2018-10-12 18:55:47.3240,67.0,San Francisco Caltrain Station 2 (Townsend St...,37.776639,-122.395526,350.0,8th St at Brannan St,37.771431,-122.405787,1095,Subscriber,No


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201458 entries, 0 to 201457
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             201458 non-null  int64  
 1   start_time               201458 non-null  object 
 2   end_time                 201458 non-null  object 
 3   start_station_id         200102 non-null  float64
 4   start_station_name       200102 non-null  object 
 5   start_station_latitude   201458 non-null  float64
 6   start_station_longitude  201458 non-null  float64
 7   end_station_id           200102 non-null  float64
 8   end_station_name         200102 non-null  object 
 9   end_station_latitude     201458 non-null  float64
 10  end_station_longitude    201458 non-null  float64
 11  bike_id                  201458 non-null  int64  
 12  user_type                201458 non-null  object 
 13  bike_share_for_all_trip  201458 non-null  object 
dtypes: f

Detect 1: start_time and end_time is not in datetime data type.

Detec 2: Ids are ideal to be in integer datatype not float. Therefore Start_station_id, end_station_id need to change the data type.

In [6]:
df.isnull().sum()

duration_sec                  0
start_time                    0
end_time                      0
start_station_id           1356
start_station_name         1356
start_station_latitude        0
start_station_longitude       0
end_station_id             1356
end_station_name           1356
end_station_latitude          0
end_station_longitude         0
bike_id                       0
user_type                     0
bike_share_for_all_trip       0
dtype: int64

Detect 3: Null values detected undert start_station_id, start_station_name, end_station_id, end_station_name.

Detect 4: Drop the rows with null values

In [7]:
df.duplicated().sum()

0

In [8]:
df.bike_share_for_all_trip.value_counts()

No     182932
Yes     18526
Name: bike_share_for_all_trip, dtype: int64

In [9]:
df.user_type.value_counts()

Subscriber    175215
Customer       26243
Name: user_type, dtype: int64

In [10]:
df['start_station_id'].duplicated().sum()

201139

In [11]:
df['end_station_id'].duplicated().sum()

201139

In [12]:
df['bike_id'].duplicated().sum()

198545

Detect 5: Remove duplicated id (?????)

### Assessment conclusion

1.

### What is the structure of your dataset? 
### What is/are the main feature(s) of interest in your dataset? 
### What features in the dataset do you think will help support your investigation into your feature(s) of interest? 


1. Change start_time and end_time data type to datetime

In [66]:
df['start_time'] = pd.to_datetime(df['start_time'])

In [67]:
df['end_time'] = pd.to_datetime(df['end_time'])

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201458 entries, 0 to 201457
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   duration_sec             201458 non-null  int64         
 1   start_time               201458 non-null  datetime64[ns]
 2   end_time                 201458 non-null  datetime64[ns]
 3   start_station_id         200102 non-null  float64       
 4   start_station_name       200102 non-null  object        
 5   start_station_latitude   201458 non-null  float64       
 6   start_station_longitude  201458 non-null  float64       
 7   end_station_id           200102 non-null  float64       
 8   end_station_name         200102 non-null  object        
 9   end_station_latitude     201458 non-null  float64       
 10  end_station_longitude    201458 non-null  float64       
 11  bike_id                  201458 non-null  int64         
 12  user_type       

In [69]:
df.start_time.sample(5)

176746   2018-10-04 09:35:42.338
191879   2018-10-02 10:49:52.342
136622   2018-10-10 16:01:39.624
5811     2018-10-31 08:51:00.054
25288    2018-10-27 23:40:38.651
Name: start_time, dtype: datetime64[ns]

In [70]:
df.end_time.sample(5)

170721   2018-10-05 07:52:54.009
61696    2018-10-22 16:44:49.163
50771    2018-10-24 07:42:44.107
43823    2018-10-24 21:20:13.321
110103   2018-10-15 08:17:00.265
Name: end_time, dtype: datetime64[ns]

3. Replace to 0 for null values under start_station_id, start_station_name, end_station_id, end_station_name.

In [14]:
df = df.dropna(how='any',axis=0) 

In [15]:
df.isnull().sum()

duration_sec               0
start_time                 0
end_time                   0
start_station_id           0
start_station_name         0
start_station_latitude     0
start_station_longitude    0
end_station_id             0
end_station_name           0
end_station_latitude       0
end_station_longitude      0
bike_id                    0
user_type                  0
bike_share_for_all_trip    0
dtype: int64

In [16]:
df.shape

(200102, 14)

3. Change Start_station_id, end_station_id, bike_id Ids datatype to integer.

In [17]:
df['start_station_id'] = pd.to_numeric(df['start_station_id'])
df['end_station_id '] = pd.to_numeric(df['end_station_id'])
df['bike_id'] = pd.to_numeric(df['bike_id'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200102 entries, 0 to 201457
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration_sec             200102 non-null  int64  
 1   start_time               200102 non-null  object 
 2   end_time                 200102 non-null  object 
 3   start_station_id         200102 non-null  float64
 4   start_station_name       200102 non-null  object 
 5   start_station_latitude   200102 non-null  float64
 6   start_station_longitude  200102 non-null  float64
 7   end_station_id           200102 non-null  float64
 8   end_station_name         200102 non-null  object 
 9   end_station_latitude     200102 non-null  float64
 10  end_station_longitude    200102 non-null  float64
 11  bike_id                  200102 non-null  int64  
 12  user_type                200102 non-null  object 
 13  bike_share_for_all_trip  200102 non-null  object 
 14  end_